In [1]:
import os, pickle, sys
import matplotlib.pyplot as plt
from scipy import stats
import numpy as np
import glob
from prettytable import PrettyTable
from tqdm import tqdm
from torch.utils.data import Dataset

import torch
from torch import nn
from torch.nn import Linear,Sequential
from torch.utils.data import DataLoader
import torch.nn.functional as func
import time
from math import isnan,isinf,log,exp
import random

In [2]:
fname=r'../myp/tensors.p'
rname='CIFAR10'

In [3]:
runs=[]
f = open(fname,'rb')
while(1):
    try:
        runs.append(pickle.load(f))
    except EOFError:
        break
f.close()
print(fname, len(runs))

../myp/tensors.p 15625


In [4]:
# len(runs)

In [5]:
# len(runs[0]['logmeasures']['synflow'])

In [6]:
# runs[0]['logmeasures']['synflow']

In [7]:
x_data=[]
y_data=[]
for idx in range(len(runs)):
    temp=runs[idx]['logmeasures']['synflow']
    for i in range(len(temp)):
        temp[i]=temp[i].item()
    temp=torch.tensor(temp,dtype=torch.float32)
    x_data.append(temp)
    y_data.append(runs[idx]['testacc'])

In [8]:
x_data=torch.tensor([item.cpu().detach().numpy() for item in x_data],dtype=torch.float64).cuda()

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10424\669201141.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at  C:\cb\pytorch_1000000000000\work\torch\csrc\utils\tensor_new.cpp:204.)
  x_data=torch.tensor([item.cpu().detach().numpy() for item in x_data],dtype=torch.float64).cuda()


In [9]:
y_data=torch.tensor(y_data,dtype=torch.float64)

In [10]:
y_data=y_data/100

In [11]:
trainx=[]
trainy=[]

In [12]:
for i in range (len(x_data)):
    if isinf(torch.sum(x_data[i]).item()):
        continue
    trainx.append(x_data[i])
    trainy.append(y_data[i])

In [13]:
trainx=torch.tensor([item.cpu().detach().numpy() for item in trainx],dtype=torch.float64)

In [14]:
trainy=torch.tensor(trainy,dtype=torch.float64)

In [15]:
print(len(trainx),len(trainy))

15404 15404


In [30]:
trainy

tensor([0.8152, 0.9051, 0.8184,  ..., 0.8944, 0.6579, 0.8401], device='cuda:0')

In [31]:
indices = torch.sort(trainy,stable=True).indices
indices=torch.sort(indices,stable=True).indices
print(indices)

tensor([ 2621, 15195,  2694,  ..., 14124,   611,  3861], device='cuda:0')


In [41]:
trainy=indices
trainy=torch.tensor(trainy,dtype=torch.float64)

C:\Users\Administrator\AppData\Local\Temp\ipykernel_10424\3018477576.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  trainy=torch.tensor(trainy,dtype=torch.float64)


In [42]:
trainx=trainx.cuda()
trainy=trainy.cuda()

In [43]:
#这句话很关键，nn默认类型为float32
torch.set_default_dtype(torch.float64)

In [142]:
class zyt(nn.Module):
    def __init__(self):
        super(zyt,self).__init__()
        self.model1=Sequential(
#           nn.BatchNorm1d(188),
            nn.LayerNorm(188),
            Linear(188,128),
#             nn.LayerNorm(128),
            nn.BatchNorm1d(128),
            nn.Sigmoid(),
            Linear(128,64),
            nn.BatchNorm1d(64),
            nn.Sigmoid(),
            Linear(64,10),
            nn.BatchNorm1d(10),
            Linear(10,1),
        )
    
    def forward(self,x):
        x=self.model1(x)
        return x
zyt1=zyt()
zyt1=zyt1.cuda()

In [143]:
#损失函数
class SpearmanLossFunc(nn.Module):
    def __init__(self):
        super(SpearmanLossFunc,self).__init__()

    def forward(self, t1,t2):
        t1=t1.cpu().detach().numpy()
        t2=t2.cpu().detach().numpy()
        loss = -abs(stats.spearmanr(t1,t2,nan_policy='omit').correlation)
        loss=torch.tensor(loss,requires_grad=True)
        return loss
    
class MyLoss(nn.Module):
    def __init__(self):
        super(MyLoss,self).__init__()

    def forward(self, t1,t2):
        t1=t1.cpu().detach().numpy()
        t2=t2.cpu().detach().numpy()
        loss=0
        sample_size=500
        sample=random.sample(range(len(trainx)),sample_size)
        t1=t1.reshape(-1,)
        for i in range(sample_size):
            for j in range(sample_size):
                if i==j:
                    continue
                loss+=log(1+exp(-np.sign((t1[sample[i]]-t1[sample[j]])*(t2[sample[i]]-t2[sample[j]]))))
        #loss=loss/(sample_size*(sample_size-1))
        loss=torch.tensor(loss,requires_grad=True)
        return loss

In [144]:
# loss_fn=torch.nn.MSELoss()#SpearmanLossFunc()
loss_fn=MyLoss()
loss_fn=loss_fn.cuda()
#优化器
learning_rate=0.001
optimizer=torch.optim.Adam(zyt1.parameters(),lr=learning_rate)
#训练的轮数
epoch=500

In [145]:
final_output=[]
for i in range(epoch):
    zyt1.train()
    print("------第{}轮训练开始-------".format(i+1))
#     start_time=time.time()
    total_train_loss=0
    output=zyt1(trainx)
    print(output)
#     print(abs(stats.spearmanr(output.cpu().detach().numpy(),trainy.cpu().detach().numpy(),nan_policy='omit').correlation))
    loss=loss_fn(output,trainy)
    #优化
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()                
    print("loss:{}".format(loss.item()))
#     end_time=time.time()
#     print("time_cost:{}".format(end_time-start_time))

------第1轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:222042.82439511403
------第2轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:218360.14279200288
------第3轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:218465.07034946707
------第4轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:214936.06462412293
------第5轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
       

loss:209188.18760129
------第39轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:218936.18760129725
------第40轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:213387.5059981979
------第41轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:219150.38874636163
------第42轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:218668.3887463758
------第43轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
  

loss:218931.34966243323
------第77轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:217356.7462272148
------第78轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:221052.38874637935
------第79轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:213659.14851734453
------第80轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:224124.14279201202
------第81轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907]

loss:219876.0531734271
------第115轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:212932.0531734157
------第116轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:219286.1427920102
------第117轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:214386.9807308726
------第118轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:216388.7462272176
------第119轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907

loss:211010.3887463547
------第153轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:219586.79103651288
------第154轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:224350.06462412936
------第155轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:223523.34966243914
------第156轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:215064.82439509334
------第157轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.

...

loss:220408.98073088692
------第343轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:216190.38874636608
------第344轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:217883.1485173514
------第345轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:221877.69569258965
------第346轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:214989.5508074813
------第347轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8

loss:217632.38874637097
------第381轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:217242.74622721426
------第382轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:215766.18760129804
------第383轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:220094.74622722395
------第384轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:216450.14279200317
------第385轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0

loss:214572.18760128983
------第419轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:215425.3496624243
------第420轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:215592.18760129515
------第421轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:220164.18760130418
------第422轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:222473.07034947193
------第423轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.

loss:220828.18760130083
------第457轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:212811.62897537692
------第458轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:223756.0646241276
------第459轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:219552.74622721982
------第460轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:217934.38874636035
------第461轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.

loss:217796.18760129757
------第495轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:217712.1876013018
------第496轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:216838.06462411545
------第497轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:217246.14279200905
------第498轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)
loss:217976.82439509427
------第499轮训练开始-------
tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.

In [146]:
abs(stats.spearmanr(output.cpu().detach().numpy(),trainy.cpu().detach().numpy(),nan_policy='omit').correlation)

0.18868626185690018

In [147]:
output

tensor([[ 1.0745],
        [-0.5619],
        [ 0.9748],
        ...,
        [ 0.8907],
        [ 0.3191],
        [ 0.0486]], device='cuda:0', grad_fn=<AddmmBackward0>)

In [148]:
trainy

tensor([ 2621., 15195.,  2694.,  ..., 14124.,   611.,  3861.], device='cuda:0')

In [91]:
indices = torch.sort(output,dim=0,stable=True).indices
indices=torch.sort(indices,dim=0,stable=True).indices
print(indices)

tensor([[13688],
        [ 1019],
        [ 5462],
        ...,
        [ 2268],
        [11551],
        [ 4429]], device='cuda:0')
